# PYPROTOLINC Demo

This notebook demonstrates how the different concepts within `pyprotolinc` can be used explaining various
customization options.

In [1]:
# we will need a bunch of import for the below
import pyprotolinc.models
import pyprotolinc.models.model_config  # this will register the built-in state models -> should be moved somewhere else so that its loaded automatically

As a first step a states model needs to be selected. We can choose between the built in states models or create a new one.

In [2]:
state_models = pyprotolinc.models.list_state_models()
state_models

{'MultiStateDisabilityStates': <enum 'MultiStateDisabilityStates'>,
 'AnnuityRunoffStates': <enum 'AnnuityRunoffStates'>,
 'MortalityStates': <enum 'MortalityStates'>}

We can look at the description of each of the built in models:

In [10]:
for state_model_name, state_model_class in state_models.items():
    print(state_model_name)
    print(state_model_class.describe())

MultiStateDisabilityStates
 A state model for a disabiility product with two disabled states. 
        - ACTIVE = 0
        - DIS1 = 1
        - DIS2 = 2
        - DEATH = 3
        - LAPSED = 4
    
AnnuityRunoffStates
 A state model consisting of two states:
        - DIS1 (=0) representing the annuity phase
        - DEATH (=1)
    
MortalityStates
 A state model with four states that can be used to model simple mortality term/perm products.

        - ACTIVE = 0
        - DEATH = 1
        - LAPSED = 2
        - MATURED = 3
    


In [4]:
state_model = state_models['MortalityStates']

In [5]:
for state in state_model:
    print(state)

MortalityStates.ACTIVE
MortalityStates.DEATH
MortalityStates.LAPSED
MortalityStates.MATURED


### Mapping the States to the Standard Output Model
The next thing we need to specify is how the state should be mapped to the standatd output model
(if there is need for that at all)